# Landsat 5 TM Reflectancia Superficie GEE

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio SHP con geoPandas

In [2]:
import os
import glob

In [3]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP'

In [4]:
import geopandas as gpd

In [5]:
# cargar un shp
zona = gpd.read_file("ZONA_geo.shp")
zona

,id,geometry
0,1,"POLYGON ((-70.14663 -12.74826, -69.84347 -12.7..."


In [6]:
# Crear un featureCollecion mediante GeoJSON
ee_zona = ee.FeatureCollection(zona.__geo_interface__)

In [7]:
type(ee_zona)

ee.featurecollection.FeatureCollection

## Coleccion de Landsat 5 TM RS

In [8]:
L5_RS = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")\
          .filterDate('2005-01-01','2009-12-31')\
          .filterMetadata('CLOUD_COVER','less_than',20)\
          .filterBounds(ee_zona)

In [10]:
# Conteo de imagenes
count = L5_RS.size().getInfo()
print("Cantidad de imagenes L5_RS:", count)

Cantidad de imagenes L5_RS: 26


In [11]:
type(count)

int

In [12]:
# Imprimir la lista de ID Imagenes
ID_L5 = L5_RS.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
print(ID_L5)

['LT05_002069_20050607', 'LT05_002069_20050709', 'LT05_002069_20050810', 'LT05_002069_20050911', 'LT05_002069_20051013', 'LT05_002069_20060712', 'LT05_002069_20060813', 'LT05_002069_20070325', 'LT05_002069_20070512', 'LT05_002069_20080903', 'LT05_002069_20090906', 'LT05_003069_20050716', 'LT05_003069_20050817', 'LT05_003069_20060516', 'LT05_003069_20060617', 'LT05_003069_20060719', 'LT05_003069_20060804', 'LT05_003069_20070823', 'LT05_003069_20070908', 'LT05_003069_20080505', 'LT05_003069_20080606', 'LT05_003069_20080825', 'LT05_003069_20080910', 'LT05_003069_20090524', 'LT05_003069_20090812', 'LT05_003069_20090828']


In [13]:
type(ID_L5)

list

In [14]:
# Crear una funcion para determinar la fecha de las imagenes ID
def list_coll(coll):
    def func_ejv(im):
        return ee.String(ee.Image(im).date().format('YYYY/MM/dd')).slice(0)
    return coll.toList(coll.size(), 0).map(func_ejv)

In [15]:
fecha = list_coll(L5_RS).getInfo()
print(fecha)

['2005/06/07', '2005/07/09', '2005/08/10', '2005/09/11', '2005/10/13', '2006/07/12', '2006/08/13', '2007/03/25', '2007/05/12', '2008/09/03', '2009/09/06', '2005/07/16', '2005/08/17', '2006/05/16', '2006/06/17', '2006/07/19', '2006/08/04', '2007/08/23', '2007/09/08', '2008/05/05', '2008/06/06', '2008/08/25', '2008/09/10', '2009/05/24', '2009/08/12', '2009/08/28']


In [16]:
type(fecha)

list

In [17]:
# Imprimir la lista Porcentaje Nubosidad
porcentaje_nube = L5_RS.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER"]).get("list").getInfo()
print(porcentaje_nube)

[6, 2, 7, 3, 3, 2, 1, 13, 14, 4, 4, 8, 9, 2, 9, 17, 5, 6, 17, 18, 15, 1, 15, 15, 11, 15]


In [18]:
# Imprimir la lista Path y Row
WRS_PATH = L5_RS.reduceColumns(ee.Reducer.toList(),["WRS_PATH"]).get("list").getInfo()
WRS_ROW = L5_RS.reduceColumns(ee.Reducer.toList(),["WRS_ROW"]).get("list").getInfo()
print(WRS_PATH)
print(WRS_ROW)

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69, 69]


In [19]:
# Crear un diccionario con las listas
dic = {"ID": ID_L5, "Fecha": fecha,"Porcentaje Nube": porcentaje_nube,"PATH": WRS_PATH,"ROW":WRS_ROW}

In [20]:
import pandas as pd

In [21]:
# Crear un DataFrame mediante el diccionario
df1 = pd.DataFrame(data = dic)
print(df1)
print("Tipo de clase: ", type(df1))

                      ID       Fecha  Porcentaje Nube  PATH  ROW
0   LT05_002069_20050607  2005/06/07                6     2   69
1   LT05_002069_20050709  2005/07/09                2     2   69
2   LT05_002069_20050810  2005/08/10                7     2   69
3   LT05_002069_20050911  2005/09/11                3     2   69
4   LT05_002069_20051013  2005/10/13                3     2   69
5   LT05_002069_20060712  2006/07/12                2     2   69
6   LT05_002069_20060813  2006/08/13                1     2   69
7   LT05_002069_20070325  2007/03/25               13     2   69
8   LT05_002069_20070512  2007/05/12               14     2   69
9   LT05_002069_20080903  2008/09/03                4     2   69
10  LT05_002069_20090906  2009/09/06                4     2   69
11  LT05_003069_20050716  2005/07/16                8     3   69
12  LT05_003069_20050817  2005/08/17                9     3   69
13  LT05_003069_20060516  2006/05/16                2     3   69
14  LT05_003069_20060617 

## Exportar tabla ID, Fecha, Porcentaje Nubosidad

In [85]:
import os

In [22]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [23]:
df1.to_csv("file_name.csv") # relative position

## Seleccion imagen Landsat 5

In [25]:
# Importar imagen Landsat 5 RS 
image = ee.Image("LANDSAT/LT05/C01/T1_SR" + "/" + ID_L5[21])

In [26]:
# Nombre de las bandas
print(image.bandNames().getInfo())

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'sr_atmos_opacity', 'sr_cloud_qa', 'pixel_qa', 'radsat_qa']


## Seleccionar Bandas

In [27]:
# Crear nombre por bandas
bandas_RS = ["B1","B2","B3","B4","B5","B7"]
bandas_TB = ["B6"]

In [28]:
# Seleccionar imagenes por bandas
img_L5_RS = image.select(bandas_RS)
img_L5_TB = image.select(bandas_TB)

In [29]:
# Nombre de las bandas
print("Banda RS:",img_L5_RS.bandNames().getInfo())
print("Banda TB:",img_L5_TB.bandNames().getInfo())

Banda RS: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7']
Banda TB: ['B6']


## Verificar proyeccion

In [30]:
# Consultar Proyeccion
print("Proyeccion RS:",img_L5_RS.projection().getInfo())
print("Proyeccion TB:",img_L5_TB.projection().getInfo())

Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 222885, 0, -30, -1336485]}
Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32619', 'transform': [30, 0, 222885, 0, -30, -1336485]}


## Metadato landsat

In [31]:
# Nombres del Metadato
img_L5_RS.propertyNames().getInfo()

['IMAGE_QUALITY',
 'SATELLITE',
 'SOLAR_AZIMUTH_ANGLE',
 'CLOUD_COVER',
 'system:id',
 'WRS_PATH',
 'EARTH_SUN_DISTANCE',
 'system:time_start',
 'LANDSAT_ID',
 'SENSING_TIME',
 'ESPA_VERSION',
 'SOLAR_ZENITH_ANGLE',
 'system:footprint',
 'system:version',
 'WRS_ROW',
 'GEOMETRIC_RMSE_MODEL_Y',
 'LEVEL1_PRODUCTION_DATE',
 'CLOUD_COVER_LAND',
 'GEOMETRIC_RMSE_MODEL_X',
 'system:asset_size',
 'GEOMETRIC_RMSE_MODEL',
 'SR_APP_VERSION',
 'PIXEL_QA_VERSION',
 'system:index',
 'system:bands',
 'system:band_names']

In [32]:
# Extraer valor metadato distancia del sol a tierra
d = img_L5_RS.get("EARTH_SUN_DISTANCE").getInfo()
d

1.010653

In [33]:
# Extraer angulo zenital solar
SOLAR_ZENITH_ANGLE = img_L5_RS.get("SOLAR_ZENITH_ANGLE").getInfo()
SOLAR_ZENITH_ANGLE

40.226799

In [34]:
import math

In [35]:
SOLAR_ZENITH_ANGLE_R = SOLAR_ZENITH_ANGLE*math.pi/180

0.7020900900879624

In [36]:
# Determinacion angulo elevacion solar radianes
SUN_ELEVATION = 90 - SOLAR_ZENITH_ANGLE
SUN_ELEVATION_R = SUN_ELEVATION*math.pi/180
print("Elevacion solar radianes:", SUN_ELEVATION_R)

Elevacion solar radianes: 0.8687062367069341


## Visualizar Landsat

In [37]:
# Simbologia estala 0 - 10000
viz_RS = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 250,
    'max': 7000,
    'gamma': 1.90
}

In [38]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [39]:
# Convertir en Geometria
geometria = ee_zona.geometry()
Map.centerObject(geometria, 8)

In [40]:
# Visualizar ESCALADO 0 - 10000
Map.addLayer(img_L5_RS, viz_RS, name = "Landsat5 RS")
Map

Map(center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['position', 'transparen…

## Factor de escala Reflectancia Superficie

In [41]:
# Escalar la imagen Landsat 5 RS 0 - 1
img_L5_RS_Es = img_L5_RS.multiply(0.0001)

In [42]:
# Simbologia escalado 0 - 1
viz_es = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [43]:
# Visualizar ESCALADO 0 - 1
Map.addLayer(img_L5_RS_Es, viz_es,name = "Landsat5 RS Escalado")
Map

Map(bottom=35438.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['positi…

## Factor de escala Temperatura Brillo

In [44]:
# Escalar la imagen Landsat 5 TB convertir Kelvin a Celsius
img_L5_TB_es = img_L5_TB.subtract(273.15).multiply(0.01)

In [45]:
# Crear su simbologia TB
viz_tb = {
    'palette': ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
                '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
                '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
                'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
                'ff0000', 'de0101', 'c21301', 'a71001', '911003'],
    'min': 25,
    'max': 28
}

In [46]:
# Visualizacion temperatura brillo
Map.addLayer(img_L5_TB_es, viz_tb,name = "TB B6 Celsius")
Map

Map(bottom=35438.0, center=[-12.908744339809369, -69.99471735110599], controls=(WidgetControl(options=['positi…

## Reproyectar y recortar segun zona

In [47]:
# Reproyectar segun zona de estudio
img_L5_TB_es = img_L5_TB_es.reproject(crs="EPSG:32719", scale=30)
img_L5_RS_Es = img_L5_RS_Es.reproject(crs="EPSG:32719", scale=30)

In [48]:
# Recortar RS y TB
img_L5_TB_es_clip = img_L5_TB_es.clip(ee_zona)
img_L5_RS_Es_clip = img_L5_RS_Es.clip(ee_zona)

In [49]:
# Consultar Proyeccion
print("Proyeccion TB:",img_L5_TB_es_clip.projection().getInfo())
print("Proyeccion RS:",img_L5_RS_Es_clip.projection().getInfo())

Proyeccion TB: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}
Proyeccion RS: {'type': 'Projection', 'crs': 'EPSG:32719', 'transform': [30, 0, 0, 0, -30, 0]}


In [50]:
img_L5_RS_Es_clip.bandNames().getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B7']

In [51]:
# Visualizacion TB y RS
Map.addLayer(img_L5_TB_es_clip, viz_tb,name = "TB B6 clip")
Map.addLayer(img_L5_RS_Es_clip, viz_es,name = "L5 RS clip")
Map

Map(bottom=17881.0, center=[-13.036669323115234, -67.00012207031251], controls=(WidgetControl(options=['positi…

## Exportar Imagen calibrado

In [52]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [53]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [54]:
# Exportar imagen Temperatura de brillo B6
geemap.ee_export_image(img_L5_TB_es_clip, filename="L5_TB_es_clip.tif", scale=30, region=geometria, file_per_band = False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L5_TB_es_clip.tif


In [55]:
# Exportar imagen Reflectancia superficie B543
geemap.ee_export_image(img_L5_RS_Es_clip.select(["B5","B4","B3"]), filename="L5_RS_Es_clip.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L5_RS_Es_clip.tif


In [56]:
# Exportar imagen Reflectancia superficie B543 Bandas separadas
geemap.ee_export_image(img_L5_RS_Es_clip.select(["B5","B4","B3"]), filename="L5_RS_Es_clip_B.tif", scale=30, region=geometria, file_per_band=True)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE


In [58]:
# Exportar en google drive
geemap.ee_export_image_to_drive(img_L5_RS_Es_clip, description='L5_RS_Es_clip', folder = 'GEE_Python', region=geometria, scale = 30)

Exporting L5_RS_Es_clip ...
